# 概要
エクアドルにある Favorita 店舗で販売されている何千もの製品フの売上を予測する。
訓練データには、日付、店舗と製品の情報、その商品がプロモーションされているか、販売数が含まれる。
追加ファイルには、モデルの構築に役立つ補足情報がある。

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Jupyter Notebook環境でのマジックコマン.グラフやプロットがNotebook上で直接表示。
%matplotlib inline
import warnings
# プログラムが実行される際に発生する警告メッセージを無視
warnings.filterwarnings('ignore')

# データの読み込み

In [2]:
df_train = pd.read_csv('data/train.csv',usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float64',},parse_dates=['date'],infer_datetime_format=True)
df_test = pd.read_csv('data/test.csv')
df_stores = pd.read_csv('data/stores.csv')
df_oil = pd.read_csv('data/oil.csv')
df_holidays = pd.read_csv('data/holidays_events.csv')
df_transactions = pd.read_csv('data/transactions.csv')

# データの分析と加工

In [6]:

print("Train Data:")
print(df_train.info())
print("\nTest Data:")
print(df_test.info())
print("\nStores Data:")
print(df_stores.info())
print("\nOil Data:")
print(df_oil.info())
print("\nHolidays Data:")
print(df_holidays.info())
print("\nTransactions Data:")
print(df_transactions.info())

Train Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   date         datetime64[ns]
 1   store_nbr    category      
 2   family       category      
 3   sales        float64       
 4   onpromotion  int64         
dtypes: category(2), datetime64[ns](1), float64(1), int64(1)
memory usage: 74.4 MB
None

Test Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB
None

Stores Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data column

データの結合とidの削除

In [10]:
df = pd.concat([df_train, df_test])
df.drop(columns='id', inplace=True) # inplace=Trueで元のdfを上書きしてしまう。

print(df.shape)
print(df.columns)

(3029400, 5)
Index(['date', 'store_nbr', 'family', 'sales', 'onpromotion'], dtype='object')


結合後のデータを確認する。

In [18]:
print("統計データ", df.describe())
print()
print("nullの確認", df.isnull().sum())
print()
print("概要", df.info())

統計データ               sales   onpromotion
count  3.000888e+06  3.029400e+06
mean   3.577757e+02  2.643830e+00
std    1.101998e+03  1.233287e+01
min    0.000000e+00  0.000000e+00
25%    0.000000e+00  0.000000e+00
50%    1.100000e+01  0.000000e+00
75%    1.958473e+02  0.000000e+00
max    1.247170e+05  7.410000e+02

nullの確認 date               0
store_nbr          0
family             0
sales          28512
onpromotion        0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3029400 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date         object 
 1   store_nbr    object 
 2   family       object 
 3   sales        float64
 4   onpromotion  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 138.7+ MB
概要 None


dateデータをdatetimeオブジェクトへ変換。
カテゴリカルデータのユニークな値数を取得。

In [20]:
# 'date' 列をdatetimeオブジェクトへ変換。（日時を扱いやすくする）
df['date'] = pd.to_datetime(df['date'])

# カテゴリデータのユニーク数を表示
for col in ['store_nbr', 'family']:
    print(f"Unique values in {col}: {df[col].nunique()}")

Unique values in store_nbr: 108
Unique values in family: 33


日付データの分解

In [23]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['week'] = df['date'].dt.day_of_week
# isinでTrueかFalseを返し、astypeで0,1へ変換する。1が土日に該当する。
df['weekend'] = df['week'].isin([5, 6]).astype(int)


count    3.029400e+06
mean     2.852941e-01
std      4.515545e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: weekend, dtype: float64